For years, I've maintained a daily practice of making single-line drawings in my sketchbooks. 

| ![sb48p072](https://i.ibb.co/TrdbwR2/sb48p072.jpg) | ![sb67p077](https://i.ibb.co/xfqZZF0/sb67p077.jpg)<br> | ![sb67p004](https://i.ibb.co/5MtbpVL/sb67p004.jpg) | ![sb38p073-gown2](https://i.ibb.co/RvJ6DpK/sb38p073-gown2.jpg) |
| -------------------------------------------------- | ------------------------------------------------------ | -------------------------------------------------- | -------------------------------------------------------------- |

When I finish a sketchbook, I scan its pages, number them, and put them into a similar folder structure. I marked which pages were diagrams/mindmaps/etc and put them in a subfolder called `notes` - the drawings (and watercolors) are in a folder called `art`. Unfortunately, I didn't tag watercolors in advance. 

![folder-list-view](https://i.ibb.co/BjQhB4t/folder-list-view.png)

Some detail on the subfolders:

- `art` subfolders contain drawings / watercolors.
- `notes` subfolders contain diagrams, mindmaps, and anything else technical I'm working on. I used to have a separate notebook for these, but I got tired of carrying around an extra sketchbook everywhere I go, so at some point they just ended up merging with my art practice.
- `xtra` contains any bad scans or funny bloopers from my scanning process. I kind of like the scans with my hands in them and thought they might be useful for a future project, so I kept them in their own folder.
- `cover` contains a scan of the notebook cover. Each cover at least has a post-it with the sketchbook ID number and the start/end dates of when I used that sketchbook. Often I'll make a little collage on the front of the notebook with airplane tickets / stickers / scraps of paper relevant to somewhere I traveled when I used that sketchbook.

<!-- 
![folder-icons](https://i.ibb.co/RD2K7SJ/folder-icons.png)
![short-sb-flipping-art2](https://i.ibb.co/6s5qxJp/short-sb-flipping-art2.gif)
-->

## The Problem: Filtering

My first problem is that the "art" sections of my sketchbooks don't just contain single-line drawings. They also contain watercolor paintings, since I don't exclusively fill my sketchbooks with single-line drawings.


<!--

| ![sb26p068-purple-hair](https://i.ibb.co/KFPv1dp/sb26p068-purple-hair.jpg) | ![sb55p069-color](https://i.ibb.co/HqsynCv/sb55p069-color.jpg) | ![sb26p098-pickle-toast](https://i.ibb.co/jW8rBNX/sb26p098-pickle-toast.jpg) | ![sb26p069-red-nose](https://i.ibb.co/ScRSctH/sb26p069-red-nose.jpg) |
| -------------------------------------------------------------------------- | -------------------------------------------------------------- | ---------------------------------------------------------------------------- | -------------------------------------------------------------------- |
-->

| ![sb69p086-wc](https://i.ibb.co/Qrf23tn/sb69p086-wc.jpg) | ![sb67p051-sparky](https://i.ibb.co/Hd4zsKb/sb67p051-sparky.jpg) | ![sb69p006](https://i.ibb.co/0Zr2MNT/sb69p006.jpg) | ![sb35p013](https://i.ibb.co/sFkp4D8/sb35p013.jpg) |
| -------------------------------------------------------- | ---------------------------------------------------------------- | -------------------------------------------------- | -------------------------------------------------- |


## Measuring Visual Similarity with Embeddings

Looking for a scalable way to filter the drawings out of my art scans, I decided to try computing visual embeddings.

<!-- I only labeled "notes" and "art" separately but didn't separate "watercolors" into their own category distinct from "drawings". The labeled data I had turned out to be useful. -->

I decided to fine-tune a vision model on my dataset to distinguish "art" pages from "notes" pages. My hope was that the model would learn to distinguish some features within the visual domain of my scanned sketchbook pages. Even though I don't intend to use the predictions of an algorithm trained to classify art-vs-notes, this will produce more meaningful embeddings.

<!-- In the past, I've found that vision models trained on standard datasets like ImageNet can produce nonsensical results when they're shown black-and-white scanned images. ImageNet consists mostly of photographs, which contain a wide distribution of colors - there aren't a lot of images in there where the majority of pixels are white. -->

Ultimately, what I want to see is that when I take the embeddings of my sketchbook pages and cluster them, the notes and drawings should neatly separate into separate clusters. Also, I want the watercolors to cleanly separate into their own clusters, away from the single-line drawings.

## Inspecting the Embeddings

After I fine-tuned my vision model, I computed the embeddings for all the sketchbook pages I had on hand. As a sanity check, I want to verify that the embedding space has cleanly separated regions for the labels is was trained on. 

It's hard to visualize 512 dimensions, so the T-SNE dimensionality reduction is a good visualization technique to view the data. T-SNE projects the higher number of dimensions into 2D or 3D, with the goal that any 2 points close together in high-dimensional space are close together in lower-dimensional space. 

I colorized the points in the resulting graph based on their label (`art`: blue, `notes`: orange, `cover`: brown) so it's easy to see at a glance that embedding space has learned something useful about this dataset, as I hoped.

![tsne-folders](https://i.ibb.co/NmMdwJS/tsne-folders.png)

## Clustering to Identify Sub-Groups

Now I'm hoping to find the watercolors in the dataset. Given that the embedding space captures visual similarity, I want to identify groups of pages that are similar to each other. This lets me label data in bulk, rather than going through the images one-by-one.

K-Means clustering is an easy tool to apply here. I can decide to ask for `K` distinct "clusters", and the algorithm picks `K` "centroids". For each point (in this case, for each sketchbook page's embedding vector), it finds the nearest centroid point. Then the algorithm adjusts the centroids iteratively, with the goal of minimizing the total distance from each point to its nearest centroid.

![Example of KMeans iterations, source: https://shabal.in/visuals/kmeans/5.html](https://shabal.in/visuals/kmeans/bottom.gif)
After some experimentation, I found that using K=16 produced groupings that would be useful to me. Each row in the image below consists of the top images from one cluster.

![all-clusters](https://i.ibb.co/Vj5kRw3/download.png)

## Using Clusters to Classify

Since each cluster's centroid is a vector in the same space as the image embeddings, I can "classify" any image embedding by doing a nearest-neighbor search of comparing the embedding of the query image to each of the cluster centroid vectors, and choosing the cluster centroid with the smallest distance. This is also called a K-Nearest Neighbor classifier.

When I applied this process to each of the images, I glanced at a t-SNE colorized by these clusters:

![t-sne all clusters](https://i.ibb.co/hsbWBGs/tsne-clusters20.png)

<!-- I've also saved the centroids, so that if I add new images I can map them to the same set of clusters. -->

## Making Sense of the Clusters

I went through and grouped these clusters based on what I wanted to identify automatically. In particular, I wanted to take only the drawings and separate them from the watercolors. After inspecting the clusters, I also realized that there were a number of bad scans / incorrectly cropped images that I wanted to exclude as well.

![drawing clusters](https://i.ibb.co/R3fg2X3/download-1.png)

![watercolor clusters](https://i.ibb.co/hYq2X4p/download-2.png)

![notes clusters](https://i.ibb.co/87L8R80/download-3.png)

![bad scans](https://i.ibb.co/Pm4t6j7/download-5.png)

![cover pages](https://i.ibb.co/k8LNmGN/download-4.png)

I went through the dataset again, and based on the cluster assigned to each image, I assigned it a higher-level group based on the hand-mapping of clusters above. 

When I looked at the t-SNE colorized by this higher-level cluster mapping, it was encouraging to see that my embedding space had neatly separated the groups I cared about:

![t-SNE higher level clusters](https://i.ibb.co/wQPXG0Q/tsne-clusters5.png)

Since my images were tagged with these higher-level groups, I was able to select only the drawings, and begin converting them into usable training data.